# Data Wraggling
In order to make good use of sematic search, we need to clean and transform data to generate embeddings before we put into vector database.

## Text Data Wranggling

We firstly clean data in `text_data.xlsx`,
 - remove useless line
 - concat content of the same

In [2]:
import pandas as pd
import numpy as np
import re

text_data_pd = pd.read_excel('data/text_data.xlsx', usecols = ['Report ID', 'Block ID', 'Text'])

mask = text_data_pd['Text'].str.startswith('<image: ')
wraggled_df_text_data = text_data_pd[~mask]

def page_num(row):
    block_value = row['Block ID'].replace(row['Report ID'], '')

    end = block_value.index('_block_')
    page_num = block_value[len('_page_'):end]
    # page_block = block_value[start:]
    # print(f'---{page_num}')
    return page_num

def block_num(row):
    # print(f'row --- {row}')
    block_value = block_value = row['Block ID'].replace(row['Report ID'], '')
    start = block_value.index('_block_')
    # end = block_value.index('_block_')
    block_num = block_value[start + len('_block_'):]
    return block_num

wraggled_df_text_data['Page'] = wraggled_df_text_data.apply(page_num, axis=1).astype('int32')
wraggled_df_text_data['Block'] = wraggled_df_text_data.apply(block_num, axis=1).astype('int32')
wraggled_df_text_data.sort_values(['Report ID', 'Page', 'Block'])

wraggled_df_text_data = wraggled_df_text_data.groupby(['Report ID'])['Text'].apply(lambda x: '\n'.join(x)).reset_index()

# date tiem pattern without leading zero
pattern = r'\d{4}\-(0?[1-9]|1[0-2])\-(0?[1-9]|1[0-9]|2[0-9]|3(0|1))_\d{1,2}\-\d{1,2}\-\d{1,2}_'

wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(pattern, "", x))
wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(r'^\d{4}\-\d{2}\-\d{2}\s', "", x))
wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(r'^\d{4}\-\d{2}\-\d{2}\s?', "", x))
wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(r'^\d{4}\-\d{2}\-\d{2}_?', "", x))
wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(r'^\d{6}\s?', "", x))
wraggled_df_text_data['Report ID'] = wraggled_df_text_data['Report ID'].map(lambda x : re.sub(r'^_', "", x))

wraggled_df_text_data.rename(columns={'Report ID': 'question', 'Text':'answers'}, inplace=True)
# wraggled_df_text_data.drop(['Report ID'], axis=1, inplace=True)
wraggled_df_text_data['manufacturing_process_number'] = 'NXT'

/tmp/ipykernel_29578/2849668654.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wraggled_df_text_data['Page'] = wraggled_df_text_data.apply(page_num, axis=1).astype('int32')
/tmp/ipykernel_29578/2849668654.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wraggled_df_text_data['Block'] = wraggled_df_text_data.apply(block_num, axis=1).astype('int32')


In [4]:
wraggled_df_text_data.drop_duplicates(inplace=True)

,question,answers,manufacturing_process_number
0,网板脏，影像相机读取Mark点错误,Printer 故障分析及解决\n故障现象：印刷时找不到Mark点.\n故障原因1：网板脏，...,NXT
1,相机镜头上表面有污点,Printer 故障分析及解决\n故障原因2：相机镜头上表面有污点.\n解决措施：移出相机，...,NXT
2,相机镜头上表面有污点,Printer 故障分析及解决\n故障原因2：相机镜头上表面有污点.\n解决措施：移出相机，...,NXT
3,PCB停落在真空托盘的位置偏移,Printer 故障分析及解决\n故障原因3： PCB停落在真空托盘的位置偏移.\n解决措施...,NXT
4,PCB有翘曲的现象,Printer 故障分析及解决\n故障原因4： PCB有翘曲的现象.\n解决措施：将PCB放...,NXT
...,...,...,...
173,Line11 EdisonCAN错误报警,VT FFEC MRN\nLine:L11\nMachine:Speedline印刷机\nM...,NXT
174,Line11 EdisonCAN错误报警,VT FFEC MRN\nLine:L11\nMachine:Speedline印刷机\nM...,NXT
175,How to find AOI & AXI Images of CFD,CFD 线AOI&AXI照片记录查询方法\n适用于预组装AXI及FAA线AXI\n适用于FA...,NXT
176,MRN-DQ200 CFD Xray 报警TUBE被关闭,VT FFEC MRN\nLine: DQ200 CFD\nMachine: Xray\nM...,NXT


## MRN20230705_2 Data Wranggling

We will group all by problems

In [162]:
mrn_pd = pd.read_excel('data/MRN20230705_2.xlsx')
mrn_pd.dtypes

LINEKEY                       int64
LINENAME                     object
MACHINEKEY                    int64
MACHINENAME                  object
MACHINETYPEKEY                int64
MACHINETYPENAME              object
PROBLEMID                     int64
PROBLEM_DES                  object
ROOT_CAUSEID                  int64
ROOT_CAUSE_DES               object
ACTIONID                      int64
ACTION_DES                   object
ACTIONID_1                    int64
REPLY_UPLOAD_FILE            object
ACTION_LINKID                 int64
UPLOAD_DATE          datetime64[ns]
UPLOAD_USERID                 int64
SHOW_SIGN                    object
dtype: object

group by problems:

In [171]:
mrn_str_pd = mrn_pd.astype(str)
duplicated_column = "PROBLEM_DES"

# Group by the 'duplicated_column' and aggregate other columns using custom functions
agg_functions = {
    'LINEKEY': lambda x: ','.join(x.unique()),
    'LINENAME': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'MACHINEKEY': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'MACHINENAME': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'MACHINETYPEKEY': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'MACHINETYPENAME': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'PROBLEMID': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ROOT_CAUSEID': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ROOT_CAUSE_DES': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ACTIONID': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ACTION_DES': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ACTIONID_1': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'REPLY_UPLOAD_FILE': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'ACTION_LINKID': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'UPLOAD_DATE': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'UPLOAD_USERID': lambda x: ','.join(x.unique()), # Example: Sum the values in 'other_column1'
    'SHOW_SIGN': lambda x: ','.join(x.unique()) # Example: Sum the values in 'other_column1'
}

# Perform the grouping and aggregation
mrn_pd_grouped = mrn_str_pd.groupby(duplicated_column).agg(agg_functions).reset_index()
mrn_pd_grouped

,PROBLEM_DES,LINEKEY,LINENAME,MACHINEKEY,MACHINENAME,MACHINETYPEKEY,MACHINETYPENAME,PROBLEMID,ROOT_CAUSEID,ROOT_CAUSE_DES,ACTIONID,ACTION_DES,ACTIONID_1,REPLY_UPLOAD_FILE,ACTION_LINKID,UPLOAD_DATE,UPLOAD_USERID,SHOW_SIGN
0,"1.能正常初始化动作. 2.生产过程中,偶尔报警CAN Node ID16探测到心搏错误 3...","10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,5558,9642,"1.电脑显示界面报NodeXX心搏错误基本判定为Can通讯异常.\n2.根据上图电路所示,C...",9559,"临时:将此线缆转弯半径最小部分移至坦克链外部,并用轧带紧固(如上图),观察两周再无报警发生....",9559,2021-3-1_14-44-19_L14 Node16 alarm.pdf,6549,2021-03-01 14:44:19,10946,Y
1,ACCELA Head Gantray X 轴移动报警，负极限,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,5059,9094,锡膏支架变形，与刮刀架干涉,8966,修复变形支架,8966,2020-3-23_11-12-48_Head Gantry X alarm.pdf,5996,2020-03-23 11:12:48,16764,Y
2,ACCELA shuttle Y报警,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,2644,"5238,5239,5240,5241,7216,8284","确定是否为驱动卡故障,传感器故障,断路器或保险故障,Shuttle Y 驱动卡故障,Shut...","5032,5033,5034,5035,7285,8470",在机器做Reset时，选中仅初始化控制器选项，观察是否出现报警信息，若出现报警信息，则说明是...,"5032,5033,5034,5035,7285,8470","2013-12-12_11-49-15_确定是否为驱动卡故障.pdf,2013-12-19_...","1717,2136,1718,1719,1720,4564,4565,5573","2013-12-12 11:49:16,2013-12-19 15:42:20,2013-1...","127,361,16764","N,Y"
3,ACCELA不识别mark点,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,4440,8142,PCB或钢网mark点异常,8351,重新制作mark点,8351,2019-3-27_14-14-33_MPMMark报警.pdf,5474,2019-03-27 14:14:33,13865,Y
4,ACCELA启动后报警Node18 stencil shuttle控制卡配置不兼容,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,4845,8837,Node18控制卡电源异常,8665,重新整理线路，紧固线路，检查插头是否开焊,8665,2019-10-29_13-40-53_MPM Node18 alarm.pdf,5738,2019-10-29 13:40:53,16764,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,机器在生产中报警硬停位未能伸展,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,3567,"6945,7751","1.由于检测PCB板是否走到位的超声波传感器功能异常引起的此报警,24伏点缺失，stoppe...","6942,7935","更换超声波传感器,断路器跳闸，检查相关负载并重新开启断路器。","6942,7935","2016-6-8_10-18-53_机器在生产中报警 硬停位未能伸展 .pdf,2018-...","4307,5103","2016-06-08 10:18:53,2018-09-15 21:07:13","623,13865",Y
62,标准MES启动流程,"10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,4188,7802,有MES报警发生需要重启MES流程，按照附件方法进行启动,"7986,8902","请按照附件流程进行启动,MES报警信息","7986,8902","2018-10-16_16-39-31_StandardMES-MPM.pdf,2020-3...","5152,5937","2018-10-16 16:39:31,2020-03-17 08:58:49","16764,361",Y
63,"生产过程中,偶尔报警Vision Gantry X Postion tracking tim...","10,214,112,52,212,1,553,574,453,373,5,653,693,...","Line 14 SMT,DQ200 BAA 1-SMT,Line 13-Old,Line 1...","2280,1911,1912,537,531,355,350,1075,2,1277,161...","14-02 MPM Accela,MPM Accela DQ200-1,MPM Accela...",33,MPM-ACCELA,5559,9643,"检查Node11相机X方向控制卡上所有连接线缆,目视无问题.\n轻拉电源线缆J4,电源线与p...",9560,更换线缆1011987,9560,2021-3-1_15-2-3_L14 Vision X alarm.pdf,6550,2021-03-01 15:02:02,10946,Y
64,自动加锡装置抖动不能置零,"1133,1222,1614,1473,1673,1373,2635,1693,2373,2353","Line 10 SMT,Line 09 SMT,Line 08 SMT,Line 07 SM...","9172,9597,9253,11912

## 将这66个问题存入文件

Combine the files and save to a single file:

In [172]:
mrn_pd_grouped['manufacturing_process_number'] = 'MPM'
mrn_pd_grouped['answers'] = "问题描述：\n" + mrn_pd_grouped['PROBLEM_DES'] + "\n问题分析：\n" + mrn_pd_grouped['ROOT_CAUSE_DES'] + "\n解决方法：\n" + mrn_pd_grouped['ACTION_DES']


columns = {
    'PROBLEM_DES':'question',
    'LINEKEY':'linekey',
    'LINENAME':'linename',
    'MACHINEKEY':'machinekey',
    'MACHINENAME':'machinename',
    'MACHINETYPEKEY':'machinetypekey',
    'MACHINETYPENAME':'machinetypename',
    'PROBLEMID':'problemid',
    'ROOT_CAUSEID':'root_causeid',
    'ROOT_CAUSE_DES':'root_cause_des',
    'ACTION_DES':'action_des',
    'ACTIONID':'actionid',
    'REPLY_UPLOAD_FILE':'reply_upload_file',
    'ACTION_LINKID':'action_linkid',
    'UPLOAD_DATE':'upload_date',
    'UPLOAD_USERID':'upload_userid',
    'SHOW_SIGN':'show_sign',
}

mrn_pd_grouped.rename(columns=columns, inplace=True)
mrn_pd_grouped.drop(['ACTIONID_1'], axis=1, inplace=True)

mrn_pd_grouped.to_csv('for-aos.csv', index=False)

## Combine data

In [212]:
combined = pd.concat([mrn_pd_grouped, wraggled_df_text_data])

In [213]:
## append excel
combined.to_csv('aos-cleaned-data.csv', index = False, header= True)

## Import to System

### Create Opensearch Index

In [316]:
# The name of index
import boto3, json
import sagemaker
import requests

sm_client = boto3.client('secretsmanager')
index_name = sm_client.get_secret_value(SecretId='opensearch-index-name')['SecretString']
data= json.loads(index_name)
index_name = data.get('index')
print(f'pre-defined index {index_name} when deployment(cdk.json)')

# Language, 'chinese' or 'english'
language = 'chinese'

# The name of embbeding model endpoint, usually you can keep it as default
eb_endpoint = 'huggingface-inference-eb'

# Ebbeding vector dimension, usually you can keep it as default
v_dimension = 768

# Docs file folder to be processed and ingested
print("Please data in 'prepared_doc' with headers ['title', 'sentence', 'paragraph']")

# Paragraph size / Chunck size
chunck_size = 200

# The imported data of the same index_name, usually you can keep it as 0 if you are creating a new index
before_import = 0

# ==============OpenSearch Related=====================
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
region = boto3.Session().region_name # e.g. cn-north-1

master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)


awsauth = (data.get('username'), data.get('password'))
url = host+'_bulk'
headers = { "Content-Type": "application/json" }

payloads = {
    "settings": {
       "index.knn": True,
        "knn.space_type": "l2"
   },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {
                        "ef_construction": 256,
                        "m": 32
                    }
                }
            },
            "answers_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {
                        "ef_construction": 256,
                        "m": 32
                    }
                }
            },
            "question": {
                "type": "text"
            },
            "answers": {
                "type": "text"
            },
            "manufacturing_process_number":{
                "type": "text"
            },
            "linekey":{
                "type": "text"
            },
            "linename":{
                "type": "text"
            },
            "machinekey":{
                "type": "text"
            },
            "machinename":{
                "type": "text"
            },
            "machinetypekey":{
                "type": "text"
            },
            "machinetypename":{
                "type": "text"
            },
            "problemid":{
                "type": "text"
            },
            "problem_des":{
                "type": "text"
            },
            "root_causeid":{
                "type": "text"
            },
            "root_cause_des":{
                "type": "text"
            },
            "actionid":{
                "type": "text"
            },
            "action_des":{
                "type": "text"
            },
            "reply_upload_file":{
                "type": "text"
            },
            "action_linkid":{
                "type": "text"
            },
            "upload_date":{
                "type": "text"
            },
            "upload_userid":{
                "type": "text"
            },
            "show_sign":{
                "type": "text"
            }
        }
    }
}

# Create Index
r = requests.put(host+index_name, auth=awsauth, headers=headers, json=payloads)

pre-defined index smart_search_qa_test when deployment(cdk.json)
Please data in 'prepared_doc' with headers ['title', 'sentence', 'paragraph']


### Remove the Opensearch Index(Optinal)

In [315]:
## You can remove the index
requests.delete(host+index_name, auth=awsauth, headers=headers)

<Response [200]>

### Load from prepared doc

Read from csv file

In [318]:
import csv
import json

# df = pd.read_csv ('for-aos.csv')
# json_data = df.to_json ()

json_array=[]
with open('aos-cleaned-data.csv', encoding = 'utf-8') as csv_file_handler:
    csv_reader = csv.DictReader(csv_file_handler)
    for row in csv_reader:
        json_array.append(row)

### Append embeddings

In [310]:
import json

endpoint_name = 'huggingface-inference-eb'
client = boto3.client('sagemaker-runtime')

def generate_vector(sentence):
    try:
        sentence = sentence if len(sentence) < 400 else sentence[:400]
        response = client.invoke_endpoint(
                        EndpointName=endpoint_name,
                        Body=json.dumps({'inputs':[sentence]}),
                        ContentType='application/json',
                    )
        vector = json.loads(response['Body'].read())
        return vector[0][0][0]
    except Exception as e:
        print(e)
        return [-1000 for _ in range(v_dimension)]


In [259]:
!pip install -q tqdm

In [319]:
from tqdm import tqdm
from time import sleep

def import_single_row(payload):
    question_vector = generate_vector(payload['question'])
    answers_vector = generate_vector(payload['answers'])

    payload['question_vector'] = question_vector
    payload['answers_vector'] = answers_vector
    first = json.dumps({ "index": { "_index": index_name} }, ensure_ascii=False) + "\n"
    second = json.dumps(payload, ensure_ascii=False) + "\n"
    payloads = first + second
    r = requests.post(url, auth=awsauth, headers=headers, data=payloads.encode()) # requests.get, post, and delete have similar syntax
        
def import_data():
    for payload in tqdm(json_array):
        import_single_row(payload)
        sleep(0.01)

In [320]:
import_data()

100%|██████████| 244/244 [00:58<00:00,  4.19it/s]
